<a href="https://colab.research.google.com/github/paul6598/AI_practice/blob/main/24.04.14_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"joonhuk","key":"e081970db4687226c0d8f799ca080829"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!kaggle datasets download -d shriyashjagtap/fraudulent-e-commerce-transactions

100% 159M/159M [00:08<00:00, 23.6MB/s]
100% 159M/159M [00:08<00:00, 20.6MB/s]


In [4]:
!unzip -o /content/fraudulent-e-commerce-transactions.zip -d /content/data_set

Archive:  /content/fraudulent-e-commerce-transactions.zip
  inflating: /content/data_set/Fraudulent_E-Commerce_Transaction_Data.csv  
  inflating: /content/data_set/Fraudulent_E-Commerce_Transaction_Data_2.csv  


In [5]:
data = pd.read_csv('/content/data_set/Fraudulent_E-Commerce_Transaction_Data.csv')

In [6]:
data = data.drop('Transaction ID', axis=1)
data = data.drop('Customer ID', axis=1)

In [7]:
data = data.drop('IP Address', axis=1)
data = data.drop('Shipping Address', axis=1)
data = data.drop('Billing Address', axis=1)

In [8]:
data = data.drop('Transaction Date', axis=1)
data = data.drop('Customer Location', axis=1)

In [9]:
data = pd.get_dummies(data)

In [10]:
data.iloc[:,6:] = data.iloc[:,6:].astype(int)

In [11]:
data

,Transaction Amount,Quantity,Customer Age,Is Fraudulent,Account Age Days,Transaction Hour,Payment Method_PayPal,Payment Method_bank transfer,Payment Method_credit card,Payment Method_debit card,Product Category_clothing,Product Category_electronics,Product Category_health & beauty,Product Category_home & garden,Product Category_toys & games,Device Used_desktop,Device Used_mobile,Device Used_tablet
0,58.09,1,17,0,30,5,0,1,0,0,0,1,0,0,0,0,0,1
1,389.96,2,40,0,72,8,0,0,0,1,0,1,0,0,0,1,0,0
2,134.19,2,22,0,63,3,1,0,0,0,0,0,0,1,0,0,0,1
3,226.17,5,31,0,124,20,0,1,0,0,1,0,0,0,0,1,0,0
4,121.53,2,51,0,158,5,0,1,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1472947,208.48,2,29,0,149,23,1,0,0,0,0,0,1,0,0,1,0,0
1472948,231.57,2,32,0,132,4,0,0,1,0,1,0,0,0,0,0,0,1
1472949,101.80,4,36,0,98,23,0,1,0,0,0,1,0,0,0,0,1,0
1472950,61.80,5,34,0,191,16,0,1,0,0,0,1,0,0,0,1,0,0


In [12]:
y = data.iloc[:,3].values
x = pd.concat([data.iloc[:,:3], data.iloc[:,4:]], axis=1)

In [13]:
y

array([0, 0, 0, ..., 0, 0, 0])

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.2)

In [19]:
model = Sequential()
model.add(Dense(10, input_dim = x_train.shape[1] ,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1, activation = 'sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 10)                180       
                                                                 
 batch_normalization_2 (Bat  (None, 10)                40        
 chNormalization)                                                
                                                                 
 dropout_3 (Dropout)         (None, 10)                0         
                                                                 
 dense_6 (Dense)             (None, 64)                704       
                                                                 
 batch_normalization_3 (Bat  (None, 64)                256       
 chNormalization)                                                
                                                                 
 dense_7 (Dense)             (None, 1)                

In [20]:
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy']) #분류는 accuracy, 회귀는 mae

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=20)
modelpath = '/content/model/best_model2.hdf5'

checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss',verbose=0, save_best_only=True)

In [21]:
history = model.fit(x_train, y_train,
                    epochs = 1000,
                    validation_split = 0.25,
                    batch_size = 64,
                    verbose=1,
                    callbacks = [early_stopping_callback,checkpointer])

Epoch 1/1000
13809/13809 [==============================] - 90s 6ms/step - loss: 0.1739 - accuracy: 0.9508 - val_loss: 0.1827 - val_accuracy: 0.9512
Epoch 2/1000
   24/13809 [..............................] - ETA: 1:02 - loss: 0.1558 - accuracy: 0.9544

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


13809/13809 [==============================] - 88s 6ms/step - loss: 0.1644 - accuracy: 0.9531 - val_loss: 0.1786 - val_accuracy: 0.9521
Epoch 3/1000
13809/13809 [==============================] - 86s 6ms/step - loss: 0.1636 - accuracy: 0.9534 - val_loss: 0.1818 - val_accuracy: 0.9512
Epoch 4/1000
13809/13809 [==============================] - 88s 6ms/step - loss: 0.1630 - accuracy: 0.9534 - val_loss: 0.1867 - val_accuracy: 0.9513
Epoch 5/1000
13809/13809 [==============================] - 87s 6ms/step - loss: 0.1627 - accuracy: 0.9535 - val_loss: 0.1887 - val_accuracy: 0.9511
Epoch 6/1000
13809/13809 [==============================] - 85s 6ms/step - loss: 0.1626 - accuracy: 0.9537 - val_loss: 0.1914 - val_accuracy: 0.9510
Epoch 7/1000
13809/13809 [==============================] - 87s 6ms/step - loss: 0.1622 - accuracy: 0.9538 - val_loss: 0.1891 - val_accuracy: 0.9512
Epoch 8/1000
13809/13809 [==============================] - 86s 6ms/step - loss: 0.1616 - accuracy: 0.9538 - val_loss: 

KeyboardInterrupt: 

In [22]:
loss, accuracy = model.evaluate(x_test, y_test)
print('오차:{:.4f}, 정확도:{:.4f}'.format(loss, accuracy))

9206/9206 [==============================] - 26s 3ms/step - loss: 0.1796 - accuracy: 0.9531
오차:0.1796, 정확도:0.9531


In [ ]:
오차:0.1524, 정확도:0.9559